# Obteniendo los datos desde un FRONTEND basico.

Levantamos y Removemos los servicios de la API(app-temp) y MongoDB(tuTiendaDB) con *docker compose*:

`docker compose up -d`

`docker compose down`

Creamos una nueva carpeta `web-page` para el `index.html`, archivos de estilos, y script basico, para hacer un fetch a la API:

```js
fetch('http://localhost:8000/client/5') 👈
       .then(function(data){
            return data.json();
     })
     .then(function(client){
         console.log(client);
          console.log(client.DOB);
          console.log(client.address);
     }) 
```

Si ejecutamos _liveserver_ desde VS CODE, dara un error en el navegador:

    _Access to fetch at 'http://localhost:8000/client/3' from origin 'http://127.0.0.1:5501' has been blocked by CORS policy: No 'Access-Control-Allow-Origin' header is present on the requested resource._

Este error es bastante comun, porque el navegador web implementa una política de seguridad llamada "Same-Origin Policy" (Política del Mismo Origen). Esta política impide que un script que se ejecuta en una página web (_http://127.0.0.1:5501_)_ haga solicitudes HTTP a un dominio, protocolo o puerto diferente (http://localhost:8000) a menos que el servidor de destino lo permita explícitamente.

Para desabilitar 'same origin policy' cierro todas las ventanas del navegador y en la terminal:

`google-chrome --disable-web-security --user-data-dir="/tmp/chrome_dev_profile"`

Ahora en el navegador se observa el FrontEnd esta Obteniendo datos del Backend:

![image](https://imgur.com/Cvvsndb.png)
![image](https://imgur.com/XcTyLat.png)


## Integrando NGINX como PROXY INVERSO

El caso anterior no es lo mas practico para el usuario. Se puede implementar un servidor **NGINX** como un proxy inverso para solucionar el problema de CORS de una manera más robusta y adecuada que deshabilitar la seguridad en el navegador.

La idea es crear una imagen lijera de nginx, como lo es: `nginx:stable-alpine`, configurar el servidor o contenedor *nginx* con un archivo llamado `nginx.conf`, y copiar todos los archivos necesarios para el FRONTEND que se encuentran en el directorio `web-page`

Nos aseguramos que el nuevo `Dockerfile` este a la altura del directorio *my-web-api*, esto es en la raiz del proyecto:

![image](https://imgur.com/izwRE9V.png)

### DOCKERFILE

```sh
FROM nginx:stable-alpine

COPY nginx.conf /etc/nginx/nginx.conf

# Copia los archivos de tu frontend al directorio de NGINX donde sirve archivos estáticos
COPY ../web-page /usr/share/nginx/html

# Expone el puerto 80 del contenedor
EXPOSE 80

# Comando para iniciar NGINX en primer plano (necesario para Docker)
CMD ["nginx", "-g", "daemon off;"]
```

### Construyendo la Imagen y el Contenedor

`docker build --tag my-nginx-proxy .`

y corremos un nuevo contenedor:

`docker run --rm --name my-nginx-container -p 80:80 --network platzinet my-nginx-proxy`

Pero hay un problema, el servidor apache ya esta corriendo por defecto en mi maquina, asi que si llego a tener problemas, simplemente se hace que se detenga:

- Para detenerlo: `sudo systemctl stop apache2`
- Para volverlo a encender(opcional despues de terminar mis pruebas): `sudo systemctl start apache2`

Tambien es muy util este comando para descubrir que proceso esta usando el puerto 80 en mi host:

`sudo ss -tuln | grep :80`





### Aspectos Claves del Archivo de Configuraciond de NGINX

```C
location /api/ {
    rewrite ^/api/(.*)$ /$1 break; 

    proxy_pass http://app-temp:8000;
    # ... otras configuraciones de proxy ...
}
```
Se configuro NGINX (`location /api/`) para que las peticiones a al API comienzen si y solo si con  `/api/`. Por tanto se modifica el codigo de JS, por ejemplo para obtener el cliente 10.

```js
fetch('http://localhost/api/client/10') 👈
       .then(function(data){
            return data.json();
     })
     .then(function(client){
         console.log(client);
          console.log(client.DOB);
          console.log(client.address);
     })  
```

Y asi se veria el FRONTEND desde la consola del navegador, se observa esta funcionando correctamente.

![](https://imgur.com/InL6zk1.png)
